In [1]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [39]:
df_train = pd.read_csv('data/df_cleanbefore10_new.csv')

df_train_orig = pd.read_csv('data/covid19_data.csv')

df_train_orig.head()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es


In [40]:
df_train.shape #(5531, 13)
df_train.head()

,target,text,length,words,hasTitle,hasContent,hasLang,hasCountry1,hasCountry2,hasCountry3,hasCountry4,num_countries
0,0.0,claim coronavirus fake belarus cope pandem hys...,98,16,0,0,0,1,1,1,1,4
1,0.0,muammar gaddafi predict current coronavirus pa...,148,23,0,0,0,1,1,1,1,4
2,0.0,video show quarantin peopl build shanghai,58,9,0,0,0,1,1,1,1,4
3,0.0,video show massiv explos wuhan china control s...,89,15,0,0,0,1,1,1,1,4
4,0.0,chlorin dioxid cure coronavirus,40,5,0,0,0,1,1,1,1,4


In [41]:
df_train['target'].value_counts(dropna=False)

0.0    5329
1.0     200
NaN       2
Name: target, dtype: int64

In [42]:
df_train['class'] = df_train_orig['class']
df_train['class'].value_counts(dropna=False)

df_train[df_train['target']==1]['class'].unique()

array(['Correct Attribution', 'True', 'Explanatory', 'Mostly True',
       'No Evidence', 'No evidence', 'News', 'no evidence', 'Half True',
       'PARTLY TRUE', 'MOSTLY TRUE', 'Partly true', 'NO EVIDENCE',
       'half true', 'HALF TRUE', 'Correct', 'HALF TRUTH',
       'Partially correct', 'True but', 'Mostly true', 'Partially true'],
      dtype=object)

In [43]:
# df_train_orig['article_source'].str.split('/',expand=True)[2].unique()

In [44]:
#function to classify the label in t(true) or f(false) or r(to revome)
def create_label(doc, list_false, list_true):
    
    if doc in list_false:
        return 'f'
    elif doc in list_true:
        return 't'
    else:
        return 'r'

#to select new data frame with the n_falses quantity of false observations
def create_balance_dataset(df,n_falses,label):
    df = df.copy()
#     df = df[['title', label]]
    df_false = df[df[label]=='f'].sample(n=n_falses, random_state=111)
    df_true  = df[df[label]=='t']
    
    return pd.concat([df_false,df_true])       

    

In [45]:
list_false= ['False', 'FALSE']
list_true= ['True','News','Mostly True','MOSTLY TRUE','HALF TRUE','Correct Attribution','Half True','PARTLY TRUE',
            'half true','Correct','Partially true','HALF TRUTH','Partially correct', 'Mostly true','True but','Partly true',
            'NO EVIDENCE','no evidence','No Evidence','No evidence']

df_train['label'] = df_train['class'].apply(create_label, args=(list_false, list_true))

df_train['label'].value_counts(dropna=False)

f    4189
r    1166
t     176
Name: label, dtype: int64

In [46]:
df_10percTrue = create_balance_dataset(df_train, 1584,'label')
print(df_10percTrue.shape)
df_10percTrue['label'].value_counts(dropna=False)

(1760, 14)


f    1584
t     176
Name: label, dtype: int64

In [47]:
df_10percTrue.head()

,target,text,length,words,hasTitle,hasContent,hasLang,hasCountry1,hasCountry2,hasCountry3,hasCountry4,num_countries,class,label
2871,0.0,african union au valid covidorgan malagasi rem...,108,14,0,0,0,1,1,1,1,4,FALSE,f
242,0.0,video muslim fruitvendor spit fruit spread cor...,75,12,0,0,0,1,1,1,1,4,FALSE,f
4218,0.0,video show alleg hotel owner new york jump win...,164,29,0,0,0,1,1,1,1,4,False,f
1602,0.0,post claim presid donald trump announc roch di...,110,16,0,0,0,1,1,1,1,4,FALSE,f
3711,0.0,imag medic facil share claim indian armi set 1...,148,24,0,0,0,1,1,1,1,4,False,f


In [48]:
df_10percTrue.drop(['class','label'],axis=1).to_csv('data/df_10percTrue_newfeatures_v2.csv', index=False)

In [49]:
df_10percTrue['hasTitle'].value_counts()

0    1667
1      93
Name: hasTitle, dtype: int64